In [1]:
!pip install datasets transformers --quiet

In [2]:
import pandas as pd

data = pd.read_csv('./data.csv')

In [3]:
from datasets import Dataset

def conv_to_ds(_data, _is_train):
    filtered_df = _data[_data['is_train']==int(_is_train)][['text', 'labels']]
    filtered_df = filtered_df.rename(columns={"labels": "labels"})
    return Dataset.from_pandas(filtered_df)

train_dataset  = conv_to_ds(data, _is_train=True)
test_dataset = conv_to_ds(data, _is_train=False)

In [4]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(
        model_name,
        num_labels=6
        )
model = AutoModelForSequenceClassification.from_config(config)

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="./tests_trainer")

In [7]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="./test_trainer", evaluation_strategy="epoch")

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 16000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6000


Epoch,Training Loss,Validation Loss


Saving model checkpoint to ./test_trainer/checkpoint-500
Configuration saved in ./test_trainer/checkpoint-500/config.json
Model weights saved in ./test_trainer/checkpoint-500/pytorch_model.bin
